## Learning XGBoost

This is lecture note [StatQuest XGBoost playlist](https://www.youtube.com/playlist?list=PLblh5JKOoLULU0irPgs1SnKO6wqVjKUsQ) on youtube.

XGBoost is implementation of gradient boosting machine (GBM).

XGBoost uses unique decision trees.

Initial prediction is 0.5 for both regression *AND* classification.

Just like other GBMs, XGBoost trains subsequent tress on the residuals.

### 1. Regression

XGBoost calculates similarity score (or quality score) for residuals for each node.
$$ \text{similarity score} = \frac{(\sum \text{residuals})^2}{(\text{number of residuals} + \lambda)} $$

where $\lambda$ is the regularization parameter. Regularization parameter lambda has a bigger impact when the number of samples in the node is small.

When residuals are similar, they do not cancel each other out and the similarity score is relatively large, and vice versa.

Similarity score is used to calculate gain of the split. Make a split and calculate similarity score for each node.

$$\text{Gain} = \text{similarity score}_\text{left} + \text{similarity score}_\text{right} - \text{similarity score}_\text{node}$$

Choose the split with biggest gain.

Continue down the split (the default is 6 levels).

After tree is built, prune the tree based on gain value.

gamma ($\gamma$) is the tree complexity parameter.

Gain has to be greater than gamma, otherwise pruned. Start from the bottom.

Setting gamma equal to zero does not equal to no pruning because lambda can make gain negative.

How does this tree make prediction?

We start output value for each sample.

$$\text{output value} = \frac{\sum \text{residuals}}{(\text{number of residuals} + \lambda)}$$

Notice the formula resembles similarity score, but doesn't have the squared term on numerator. **$\lambda$ shrinks similarity score AND output value**.

The final prediction is initial prediction (0.5) + learning rate * output value for the first tree + learning rate * output value for the second tree + ...

where learning Rate (eta) default = 0.3.

### 2. Classification

Classification has slightly different similarity score.

$$ \text{similarity score} = \frac{(\sum \text{residuals})^2}{\sum[\text{previous probability} \cdot (1 - \text{previous probability})] + \lambda} $$

The minimum number of residuals in each leaf is determined by **cover**.

$$\text{Cover} = \text{denominator of the similarity score} - \lambda$$
$$= \sum[\text{previous probability} * (1- \text{previous probability})]$$

For regression, cover = number of residuals.

If the cover of the leaf is less than the minimum number of cover (default 1), XGBoost will prune that leaf.

By default, the minimum number of cover is 1 is equivalent to `min_child_weight` parameter of XGBoost.

Output value of classification is also different.

$$\text{output value} = \frac{\sum \text{residuals}}{\sum[\text{previous probability} * (1- \text{previous probability})] + \lambda}$$

### 3. Math

### 4. Optimizations

Note: the decision to use the threshold that gives the largest gain is made without worrying about how the leaves will be split later and that means XGBoost uses greedy algorithm to build trees. In other words, since XGBoost uses a greedy algorithm, it makes a decision without looking ahead to see if it is the absolute best choice in the long term.

By using a greedy algorithm, XGBoost can build a tree relatively quickly.

For XGBoost, the approximate greedy algorithm means that instead of testing all possible thresholds, we only test the quantiles (default around 33 quantiles).

Weighted quantile

The weights are derived from the cover metric. For regression, since cover is 1 for each observation, weighted quantiles are just like normal quantiles and contain equal number of observations.